In [1]:
from keras.models import Sequential, load_model
from keras.layers import *
from keras import optimizers
from keras.callbacks import TensorBoard

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

import tensorflow as tf

# Importing matplotlib to plot images.
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# Importing SK-learn to calculate precision and recall
import sklearn
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score, LeaveOneGroupOut
from sklearn.utils import shuffle 

# Used for graph export
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
from keras import backend as K

import pickle as pkl
import h5py

from pathlib import Path
import os.path
import sys
import datetime
import time
import json

Using TensorFlow backend.


In [2]:
# If GPU is not available: 
# GPU_USE = '/cpu:0'
# config = tf.ConfigProto(device_count = {"GPU": 0})


# If GPU is available: 
config = tf.ConfigProto()
config.log_device_placement = True
config.allow_soft_placement = True
config.gpu_options.allocator_type = 'BFC'

# Limit the maximum memory used
config.gpu_options.per_process_gpu_memory_fraction = 0.1

# set session config
tf.keras.backend.set_session(tf.Session(config=config))

In [3]:
class LoggingTensorBoard(TensorBoard):    

    def __init__(self, log_dir, settings_str_to_log, **kwargs):
        super(LoggingTensorBoard, self).__init__(log_dir, **kwargs)

        self.settings_str = settings_str_to_log

    def on_train_begin(self, logs=None):
        TensorBoard.on_train_begin(self, logs=logs)

        tensor =  tf.convert_to_tensor(self.settings_str)
        summary = tf.summary.text ("Run Settings", tensor)

        with  tf.Session() as sess:
            s = sess.run(summary)
            self.writer.add_summary(s)
        

In [4]:
smartphones = ["N5X"]
# TODO remove
participants = list(range(1, 4))
window_size = 20
constant_pixels = 0.06
HDF5_PATH = str(Path.home())+"/data/hdf/"+smartphones[0]+"-win"+str(window_size)+".hdf"

# H5DF files contains 3 members: labels, segments, groups

In [5]:
hdf = h5py.File(HDF5_PATH, "r")

train_x = hdf["train/sensors"]
train_y = hdf["train/labels"]

test_x = hdf["test/sensors"]
test_y = hdf["test/labels"]

print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

(343296, 20, 18, 1) (343296, 2) (154389, 20, 18, 1) (154389, 2)


In [6]:
def myGenerator(set_name, batch_size):
    """
    This generator returns images
    """
    hdf = h5py.File(HDF5_PATH, "r")

    pImages = hdf[set_name + "/sensors"]
    pLabels = hdf[set_name + "/labels"]

    len_train = pImages.shape[0]
    
    randomBatchOrder = np.arange(len_train // batch_size)
       
    while True:
        np.random.shuffle(randomBatchOrder) 
        
        for i in range(len_train // batch_size):
            idx = randomBatchOrder[i]
            shuffled = shuffle(pImages[idx * batch_size: (idx+1) * batch_size], pLabels[idx * batch_size: (idx+1) * batch_size])
            yield shuffled[0].reshape(-1, 1, 20, 18, 1), shuffled[1].reshape(-1, 2, 1)

## Split Training- & Test-Data

In [7]:
## ConvLSTM net hyperparameters
numChannels = 1
numFilters = 128 # number of filters in Conv2D layer
# kernal size of the Conv2D layer
kernalSize1 = (3,3)
# max pooling window size
poolingWindowSz = 2
# number of filters in fully connected layers
numNueronsFCL1 = 512
numNueronsFCL2 = 128
# number of epochs
epochs = 9999
# batchsize
batch_size = 100
# number of total clases
numClasses = train_y.shape[1]
# dropout ratio for dropout layer
dropOutRatio = 0.2

In [8]:
y = "" 

In [ ]:
tf.get_default_graph()

model = None
model = Sequential()
# adding the first convLSTM layer with 32 filters and 5 by 5 kernal size, using the rectifier as the activation function
model.add(ConvLSTM2D(numFilters, kernalSize1 ,input_shape=(None, train_x.shape[1], train_x.shape[2], 1),activation='relu', padding='same',return_sequences=True))

## adding a maxpooling layer
model.add(TimeDistributed(MaxPooling2D(pool_size=(poolingWindowSz,poolingWindowSz),padding='valid')))

## adding a dropout layer for the regularization and avoiding over fitting
model.add(Dropout(dropOutRatio))

## flattening the output in order to apple dense layer
model.add(TimeDistributed(Flatten()))

## adding first fully connected layer with 256 outputs
model.add(Dense(numNueronsFCL1, activation='relu'))

## adding second fully connected layer 128 outputs
model.add(Dense(numNueronsFCL2, activation='relu'))

## flattening the output in order to apply the fully connected layer
model.add(TimeDistributed(Flatten()))

## adding softmax layer for the classification
model.add(Dense(numClasses))

## Compiling the model to generate a model
def eucInMM(y_true, y_pred):
    return (K.sqrt(K.sum(K.square(y_true - y_pred), axis=-1, keepdims=True))) * constant_pixels
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 

def euc(y_true, y_pred):
    return K.sqrt(K.sum(K.square(y_true - y_pred), axis=-1, keepdims=True))

#optimizer = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
#optimizer = optimizers.Adam(lr = 0.001, decay=1e-6)
optimizer = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.1)
model.compile(loss=rmse, optimizer=optimizer, metrics=[eucInMM ,rmse, euc])

func = eval('model.summary(print_fn = lambda x: y + x)')
print(func)

# Broadcast progress to the tensorboard. 
readable_timestamp = datetime.datetime.fromtimestamp(time.time()).strftime('%Y%m%d_%H%M%S')
tensorflowFolder = "/srv/share/tensorboardfiles/IMU_B(" + str(batch_size)+ ")_K" + str(kernalSize1)+"_" + smartphones[0] + "_" + readable_timestamp
#model.summary()

config = ""
for layer in model.layers:
    config += str(layer.output).split('\"')[1].split("/")[0] + str(layer.output_shape) + "\n\n"
model.summary()

history = model.fit_generator(myGenerator("train", batch_size),
                    steps_per_epoch=len(train_x) // batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=myGenerator("test", batch_size),
                    validation_steps=len(test_x) // batch_size,
                    callbacks=[LoggingTensorBoard(settings_str_to_log = config, log_dir=tensorflowFolder,
                                                  histogram_freq=0, write_graph=True, write_images=True, update_freq = 'epoch')])

score = model.evaluate(np.expand_dims(testX,1),np.expand_dims(testY,1),verbose=2)
print('%s: %.2f' % (model.metrics_names[1], score[1]))
print('Baseline ConvLSTM Error: %.2f' %(1 - score[1]))

None
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_5 (ConvLSTM2D)  (None, None, 20, 18, 128) 594944    
_________________________________________________________________
time_distributed_13 (TimeDis (None, None, 10, 9, 128)  0         
_________________________________________________________________
dropout_5 (Dropout)          (None, None, 10, 9, 128)  0         
_________________________________________________________________
time_distributed_14 (TimeDis (None, None, 11520)       0         
_________________________________________________________________
dense_13 (Dense)             (None, None, 512)         5898752   
_________________________________________________________________
dense_14 (Dense)             (None, None, 128)         65664     
_________________________________________________________________
time_distributed_15 (TimeDis (None, None, 128)         0         
_____